<a href="https://colab.research.google.com/github/DIPANJAN001/Dipanjanpatra/blob/master/Hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Hyperparameter Optimization

In [3]:
import pandas as pd
weather_data=pd.read_excel("/content/pv_02.xlsx") 

In [4]:
weather_input1=weather_data.drop('power_normed',axis=1)
weather_input=weather_input1.drop('time_idx',axis=1)
weather_input.head()

,hour_of_day,hour_of_day_cos,hour_of_day_sin,month_of_year,month_of_year_cos,month_of_year_sin,season_of_year,season_of_year_cos,season_of_year_sin,sunposition_thetaZ,...,LowerWindSpeed,LowerWindDirection,LowerWindDirectionMath,LowerWindDirectionCos,LowerWindDirectionSin,UpperWindSpeed,UpperWindDirection,UpperWindDirectionMath,UpperWindDirectionCos,UpperWindDirectionSin
0,0.000000,1.000000,0.000000,0.909091,0.017332,0.258819,1.0,0.0,0.0,1.000000,...,0.458204,0.554229,0.195738,0.667024,0.971278,0.494272,0.563420,0.186512,0.694226,0.960735
1,0.142857,0.957824,0.399332,0.909091,0.017332,0.258819,1.0,0.0,0.0,1.000000,...,0.432281,0.726648,0.023317,0.994589,0.573364,0.456086,0.732230,0.017730,0.996887,0.555710
2,0.285714,0.838278,0.732544,0.909091,0.017332,0.258819,1.0,0.0,0.0,1.000000,...,0.513296,0.702088,0.047877,0.977444,0.648484,0.545543,0.707816,0.042140,0.982548,0.630947
3,0.428571,0.661157,0.944463,0.909091,0.017332,0.258819,1.0,0.0,0.0,0.705699,...,0.676684,0.691446,0.058519,0.966460,0.680042,0.663653,0.693619,0.056335,0.968971,0.673395
4,0.571429,0.455788,1.000000,0.909091,0.017332,0.258819,1.0,0.0,0.0,0.611994,...,0.656101,0.698192,0.051774,0.973667,0.660124,0.634276,0.700475,0.049480,0.976001,0.653047


In [6]:
solpow=weather_data['power_normed']
solpow

0       0.000000
1       0.000000
2       0.000000
3       0.014560
4       0.152064
          ...   
6273    0.110155
6274    0.254311
6275    0.003055
6276    0.000000
6277    0.000000
Name: power_normed, Length: 6278, dtype: float64

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(weather_input,solpow, test_size=0.25, random_state=42,shuffle=False)

In [9]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

In [68]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
def create_model():
      model = Sequential()
      model.add(Dense(8, kernel_initializer='normal',input_dim =X_train.shape[1], activation='relu'))


      model.add(Dense(32, kernel_initializer='normal',activation='relu'))
      model.add(Dense(64, kernel_initializer='normal',activation='relu'))
      model.add(Dense(16, kernel_initializer='normal',activation='relu'))

      model.add(Dense(1, kernel_initializer='normal',activation='linear'))


      model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
      return model
#MLP_model.summary()

In [69]:
model = KerasRegressor(build_fn=create_model, verbose=0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


In [70]:
batch_size = [16,32,64]
epochs = [80,100,120]

In [ ]:
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

In [27]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: -0.029987 using {'batch_size': 64, 'epochs': 80}


In [28]:
mae = grid_result.cv_results_['mean_test_score']
params = grid_result.cv_results_['params']
for mae, param in zip(mae, params):
    print("%f  with: %r" % (mae,param))

-0.030401  with: {'batch_size': 16, 'epochs': 80}
-0.035140  with: {'batch_size': 16, 'epochs': 100}
-0.032478  with: {'batch_size': 16, 'epochs': 120}
-0.031507  with: {'batch_size': 32, 'epochs': 80}
-0.030867  with: {'batch_size': 32, 'epochs': 100}
-0.031536  with: {'batch_size': 32, 'epochs': 120}
-0.029987  with: {'batch_size': 64, 'epochs': 80}
-0.030425  with: {'batch_size': 64, 'epochs': 100}
-0.030965  with: {'batch_size': 64, 'epochs': 120}
